In [2]:
import pandas as pd
import random
import math
#from penalty_cost import calculate_penalty_cost
import copy
df=pd.read_excel('Line Production September 2023.xlsx')

#  %run penalty_cost.ipynb

In [3]:
def calculate_penalty_cost(df, solution):
    total_penalty_cost = 0

    for line, products in solution.items():

        #initial values 
        line_penalty_cost = 0  
        current_time = 0  

        for product in products:
            #get the data of the specific product that's in the current line 
            product_row = df[df['Product'] == product]

            if not product_row.empty:

                # the products processing time on the line it's being processed on 
                processing_time = product_row[line].values[0]
                # product deadline 
                deadline = product_row['deadline'].values[0]

                # how late the product is 
                lateness = current_time + processing_time - deadline

                #check if the product is late and if yes what the cost is     
                if lateness > 0:
                    penalty_cost = lateness * product_row['penalty cost'].values[0]
                    #add the penalty cost to total cost of that line
                    line_penalty_cost += penalty_cost

                #change current time to that after processing the product 
                current_time += processing_time

        # add penalty cost of everyline together 
        total_penalty_cost += line_penalty_cost

    return total_penalty_cost

In [6]:
initial_temperature = 10
cooling_rate = 0.995
iterations_per_temperature = 100

# Simulated annealing function
def simulated_annealing(df, schedule, temperature, cooling_rate, iterations_per_temp):
    current_schedule = schedule
    best_schedule = schedule
    current_cost = calculate_penalty_cost(df, current_schedule)
    best_cost = current_cost

    while temperature > 0.1:
        for _ in range(iterations_per_temp):
            # Generate a neighboring solution by swapping two random cities
            i, j = random.sample(list(initial_solution.keys()), 2)
            #neighbor_schedule = current_schedule[:]
            neighbor_schedule = copy.deepcopy(current_schedule)
            neighbor_schedule[i], neighbor_schedule[j] = neighbor_schedule[j], neighbor_schedule[i]
            neighbor_cost = calculate_penalty_cost(df, neighbor_schedule)

            # Decide whether to accept the neighbor solution
            delta = neighbor_cost - current_cost
            if delta < 0 or random.random() < math.exp(-delta / temperature):
                current_schedule = neighbor_schedule
                current_cost = neighbor_cost

                # Update the best solution if necessary
                if current_cost < best_cost:
                    best_schedule = current_schedule
                    best_cost = current_cost

        # Cool the temperature
        temperature *= cooling_rate

    return best_schedule, best_cost

initial_solution = {"L1": ['P26', 'P17', 'P18', 'P20'],
"L2": ['P10', 'P7', 'P27', 'P19', 'P12'],
"L3": ['P13', 'P14', 'P22', 'P15'],
"L4": ['P11', 'P9', 'P25'],
"L5": ['P23', 'P16', 'P21'],
"L6": ['P2', 'P5', 'P8', 'P1'],
"L7": ['P6', 'P4', 'P24', 'P3']}



best_schedule, best_cost = simulated_annealing(df, initial_solution, initial_temperature, cooling_rate, iterations_per_temperature)

print("Best Schedule:", best_schedule)
print("Best Cost:", best_cost)

Best Schedule: {'L1': ['P6', 'P4', 'P24', 'P3'], 'L2': ['P10', 'P7', 'P27', 'P19', 'P12'], 'L3': ['P13', 'P14', 'P22', 'P15'], 'L4': ['P11', 'P9', 'P25'], 'L5': ['P23', 'P16', 'P21'], 'L6': ['P2', 'P5', 'P8', 'P1'], 'L7': ['P26', 'P17', 'P18', 'P20']}
Best Cost: 14000


14000